In [114]:
# %%capture
# !pip install boto3

In [1]:
import boto3
import random
import os
from dotenv import load_dotenv
import time

load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.getenv('REGION_NAME')
AGENT_ID = os.getenv('AGENT_ID')
AGENT_ALIAS_ID = os.getenv('AGENT_ALIAS_ID')

bedrock_agent_runtime = boto3.client(
                'bedrock-agent-runtime',
                aws_access_key_id = AWS_ACCESS_KEY_ID,
                aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                region_name= REGION_NAME
            )
session = random.randint(0,9999) #setting a random sessionID

## Below shows how to call the agents that are created in AWS bedrock

In [2]:
## Function for Invoking the agent (lead_gen) that is created in AWS bedrock

def generate_response_query(input_text,agent_id,agent_alias_id,session_id):
  response = bedrock_agent_runtime.invoke_agent(inputText=input_text,
      agentId=agent_id,
      agentAliasId=agent_alias_id,
      sessionId=session_id,
  )

  print(response.get('completion'))

  # Response returned from the agent

  event_stream = response.get('completion')

  for event in event_stream:
    if 'chunk' in event:
      data = event['chunk']['bytes']
      print(f"Final answer ->\n{data.decode('utf8')}")

In [3]:
input_text:str = "Mahesh Yadav, California, 339111222333" # replace this with a prompt relevant to your agent

input_text2:str = "Calculate solar potential for 425 stewart dr mountain view ca 94043, with a monthly bill of $765"
agent_id:str = AGENT_ID # note this from the agent console on Bedrock
agent_alias_id:str = AGENT_ALIAS_ID # fixed for draft version of the agent
session_id:str =  str(session) # random identifier

In [4]:
generate_response_query(input_text2,agent_id,agent_alias_id,session_id)

Final answer ->
The solar potential analysis for the address 425 stewart dr mountain view ca 94043 with a monthly electricity bill of $765 is:

- The remaining lifetime utility bill without solar is $11891
- The estimated federal incentive is $9872
- The solar system would generate 98.4% of your electricity needs
- With a cash purchase, the out of pocket cost is $37969, the payback period is 4.5 years, and the 20 year lifetime savings is $138184
- With a financed purchase, the 20 year lifetime savings is $93573
- With a lease, the annual lease cost is $2230, and the 20 year lifetime savings is $93573

The analysis shows installing solar at this location is financially viable through purchase or lease.


In [5]:
time.sleep(4)
generate_response_query(input_text,agent_id,agent_alias_id,session_id)

Final answer ->
I have created a lead with the following information provided:

Name: Mahesh Yadav
Phone Number: 339111222333  
Address: California


## Creating an Agent and Invoking the Lambda functions stored in AWS

In [6]:
# Note: Here bedrock-agent is used instead of bedrock-agent-runtime
# Bedrock-agent has create_agent() function, while bedrock-agent-runtime has invoke_agent()

bedrock_agent = boto3.client(
                'bedrock-agent',
                aws_access_key_id = AWS_ACCESS_KEY_ID,
                aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                region_name= REGION_NAME
            )

In [7]:
aws_agent = bedrock_agent.create_agent(
    agentName='Lead_gen_notebook',
    description='Extract name, phone number and address for lead generation. Or extract address and monthly bill for calculating solar potential',
    foundationModel='anthropic.claude-v2:1',
    instruction="""You are a solar panel financial advisor, your job is to extract address and monthly bills for calling functions to calculate solar potential.The assistant should summarize the solar potential calculation from JSON format to readable text, add a statement in the end as "Please provide you name, address and phone number for contacting you" and return it. If the user provides the name, address and phone number, extract them for creating leads. You also are a solar panel lead generation employee, your job is to extract personal details like name, phone number and address for returning as response for calling functions to generate leads. Strictly respond to the question for calling the function that is required to answer. """,
    agentResourceRoleArn='arn:aws:iam::674564987265:role/service-role/AmazonBedrockExecutionRoleForAgents_KT4F6CHPJV'
)

In [8]:
time.sleep(4)
#Creating the action group for the agent
response = bedrock_agent.create_agent_action_group(
    agentId=aws_agent['agent'].get('agentId'),
    agentVersion="DRAFT",
    actionGroupName='LeadGenAction',
    actionGroupExecutor={
        'lambda': 'arn:aws:lambda:us-east-1:674564987265:function:lead_gen_solar'
    },
    apiSchema={
        's3': {
            's3BucketName': 'legalgraphbucket',
            's3ObjectKey': 'lead_generation.json'
        }
    },
    actionGroupState='ENABLED'
)
print(response)

{'ResponseMetadata': {'RequestId': 'c1cbf239-22c3-4477-8e93-1fafd98e9bf2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 27 Jan 2024 09:10:47 GMT', 'content-type': 'application/json', 'content-length': '439', 'connection': 'keep-alive', 'x-amzn-requestid': 'c1cbf239-22c3-4477-8e93-1fafd98e9bf2', 'x-amz-apigw-id': 'SMRHjEZnoAMEgSw=', 'x-amzn-trace-id': 'Root=1-65b4c896-5c7874b43dbb68aa254ff16a'}, 'RetryAttempts': 0}, 'agentActionGroup': {'agentId': 'ZC2IALTGLS', 'agentVersion': 'DRAFT', 'actionGroupId': '3H2NBA2ECP', 'actionGroupName': 'LeadGenAction', 'createdAt': datetime.datetime(2024, 1, 27, 9, 10, 47, 81348, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2024, 1, 27, 9, 10, 47, 81348, tzinfo=tzutc()), 'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:674564987265:function:lead_gen_solar'}, 'apiSchema': {'s3': {'s3BucketName': 'legalgraphbucket', 's3ObjectKey': 'lead_generation.json'}}, 'actionGroupState': 'ENABLED'}}


In [9]:
# Check status of the agent

response = bedrock_agent.get_agent(
    agentId=aws_agent['agent'].get('agentId')
)
print(response['agent'].get('agentStatus'))

NOT_PREPARED


In [10]:
#Preparing the agent

response = bedrock_agent.prepare_agent(
    agentId=aws_agent['agent'].get('agentId')
)

In [11]:
time.sleep(4)
# Check status of the agent again

response = bedrock_agent.get_agent(
    agentId=aws_agent['agent'].get('agentId')
)
print(response['agent'].get('agentStatus'))

PREPARED


In [12]:
# Creating the alias
agent_alias = bedrock_agent.create_agent_alias(
    agentId=aws_agent['agent'].get('agentId'),
    agentAliasName='Dummy',
)
print(agent_alias)

{'ResponseMetadata': {'RequestId': 'a4bd2847-9e80-41ac-b7ba-aa16f32eba2e', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 27 Jan 2024 09:13:15 GMT', 'content-type': 'application/json', 'content-length': '329', 'connection': 'keep-alive', 'x-amzn-requestid': 'a4bd2847-9e80-41ac-b7ba-aa16f32eba2e', 'x-amz-apigw-id': 'SMRezFVeoAMEdJg=', 'x-amzn-trace-id': 'Root=1-65b4c92b-5292faac6b8a2fb626ee2d42'}, 'RetryAttempts': 0}, 'agentAlias': {'agentId': 'ZC2IALTGLS', 'agentAliasId': '5WKU8ATYUM', 'agentAliasName': 'Dummy', 'agentAliasArn': 'arn:aws:bedrock:us-east-1:674564987265:agent-alias/ZC2IALTGLS/5WKU8ATYUM', 'routingConfiguration': [], 'createdAt': datetime.datetime(2024, 1, 27, 9, 13, 15, 303053, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2024, 1, 27, 9, 13, 15, 303053, tzinfo=tzutc()), 'agentAliasStatus': 'CREATING'}}


In [13]:
time.sleep(4)
#Preparing the agent

response = bedrock_agent.prepare_agent(
    agentId=aws_agent['agent'].get('agentId')
)

In [14]:
input_text:str = "Mahesh Yadav, California, 339111222333" # replace this with a prompt relevant to your agent

input_text2:str = "Calculate solar potential for 9813 Sherborne Ave, Bakersfield, CA 93311, USA with monthly bill as 300 USD"


agent_id:str = aws_agent['agent'].get('agentId') # note this from the agent console on Bedrock
agent_alias_id:str = agent_alias['agentAlias'].get('agentAliasId') # fixed for draft version of the agent
session_id:str =  str(session) # session ID

In [15]:
time.sleep(4)
generate_response_query(input_text2,agent_id,agent_alias_id,session_id)

Final answer ->
The solar potential analysis for 9813 Sherborne Ave, Bakersfield, CA 93311, USA with a $300 monthly electricity bill is:

Solar Percentage: 99.15%
Percentage Exported to Grid: 66.36%  
Lifetime Utility Bill Savings: $7857
Federal Incentive: $6837

Cash Purchase:
Payback Period: 5.25 years
Lifetime Savings: $82,126

Leasing:  
Annual Leasing Cost: $1544.95  
Lifetime Savings: $51,227

Financed Purchase:
Annual Loan Payment: $1544.95
Lifetime Savings: $51,227

Please provide you name, address and phone number for contacting you


In [16]:
time.sleep(4)
generate_response_query(input_text,agent_id,agent_alias_id,session_id)

Final answer ->
Lead created successfully for Mahesh Yadav from California with phone number 339111222333. Details have been recorded.
